In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, sum, array_contains, array, array_intersect, split
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import input_file_name, regexp_extract
from google.cloud import storage
import numpy as np
from pyspark.sql.functions import from_unixtime, to_timestamp, date_format
import io
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.cloud import storage
from google.oauth2 import service_account

# Send data from Drive to Google Cloud Storage

In [2]:
credentials = {
  "type": "service_account",
  "project_id": "cedar-pottery-388916",
  "private_key_id": "It is private...",
  "private_key": "It is private...",
  "client_email": "drive-895@cedar-pottery-388916.iam.gserviceaccount.com",
  "client_id": "113489151660006131618",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/drive-895%40cedar-pottery-388916.iam.gserviceaccount.com",
  "universe_domain": "googleapis.com"
}

credentials = service_account.Credentials.from_service_account_info(credentials)
drive_service = build('drive', 'v3', credentials=credentials)

# Define the project ID
project_id = 'cedar-pottery-388916'

# Define the destination bucket name in Google Cloud Storage
bucket_name = 'bucket123321'

# Yelp

In [3]:
# Define a list of file URLs
file_urls = [
    'https://drive.google.com/file/d/12m9N4KF4-_VkSO0QB2tHbnSuuC20L_i4',
    'https://drive.google.com/file/d/1mwNNdOMSNty6WumYdH9FJNJZJYQ6oD1c',
    'https://drive.google.com/file/d/1TT4ARRIV6i2fO1b5yb0aSUkjhxMb9u6g'
]

# Define the corresponding file names
file_names = ['business.csv', 'review.json', 'user.parquet']

# Create a client for Google Cloud Storage directly using the credentials
client = storage.Client(credentials=credentials, project=project_id)

# Iterate over the file URLs and names simultaneously
for file_url, file_name in zip(file_urls, file_names):
    # Extract the file ID from the URL
    file_id = file_url.split('/')[-1]

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)
    file_stream = io.BytesIO()
    downloader = MediaIoBaseDownload(file_stream, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()

    # Upload the file to Google Cloud Storage
    blob = client.bucket(bucket_name).blob(file_name)
    file_stream.seek(0)  # Reset the stream position to the beginning
    blob.upload_from_file(file_stream, content_type='application/octet-stream')

    print('File transfer completed for file ID:', file_id)

print('All files transferred.')

File transfer completed for file ID: 12m9N4KF4-_VkSO0QB2tHbnSuuC20L_i4
File transfer completed for file ID: 1mwNNdOMSNty6WumYdH9FJNJZJYQ6oD1c
File transfer completed for file ID: 1TT4ARRIV6i2fO1b5yb0aSUkjhxMb9u6g
All files transferred.


# Google Maps

## Metadata

In [4]:
def get_directory_and_file_info(service, directory_id, parent_directory_name=""):
    file_names = []
    file_ids = []
    file_modified_dates = []

    # Query for files in the current directory
    query = f"'{directory_id}' in parents"
    response = service.files().list(q=query).execute()

    # Iterate over the files in the current directory
    for file in response.get('files', []):
        if file['mimeType'] == 'application/vnd.google-apps.folder':
            # If the file is a subdirectory, recursively call the function
            directory_name = parent_directory_name + file['name'] + "/"
            names, ids, modified_dates = get_directory_and_file_info(service, file['id'], directory_name)
            file_names.extend(names)
            file_ids.extend(ids)
            file_modified_dates.extend(modified_dates)
        else:
            # If the file is not a subdirectory, add the combined name and ID to the lists
            combined_name = parent_directory_name + file['name']
            file_names.append(combined_name)
            file_ids.append(file['id'])

            # Retrieve the file metadata to get the modified date
            file_metadata = service.files().get(fileId=file['id'], fields='modifiedTime').execute()
            modified_date = file_metadata.get('modifiedTime', '')
            file_modified_dates.append(modified_date)

    return file_names, file_ids, file_modified_dates

# ID of the top-level directory
top_directory_id = '1wGNQYA3f_PJpUkarEes0txvYpQ2k1ctB'

# Call the function to retrieve directory and file names, IDs, and modified dates recursively
names, ids, modified_dates = get_directory_and_file_info(drive_service, top_directory_id)

# Print the directory and file names, IDs, and modified dates
for name, file_id, modified_date in zip(names, ids, modified_dates):
    print("Name:", name)
    print("ID:", file_id)
    print("Last Modified Date:", modified_date)
    print()

# Alternatively, you can access the directory and file names, IDs, and modified dates separately as lists
print("Directory and File Names:", names)
print("File IDs:", ids)
print("Last Modified Dates:", modified_dates)

# -----------------------------------------------------------------------------------------

# Create a client for Google Cloud Storage directly using the credentials
client = storage.Client(credentials=credentials, project=project_id)

# Define the target directory path within the bucket
target_directory = "gmap_metadata/"

# Iterate over the file URLs and names simultaneously
for id, file_name in zip(ids, names):

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=id)
    file_stream = io.BytesIO()
    downloader = MediaIoBaseDownload(file_stream, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()

    # Upload the file to Google Cloud Storage
    target_blob_name = target_directory + file_name
    blob = client.bucket(bucket_name).blob(target_blob_name)
    file_stream.seek(0)  # Reset the stream position to the beginning
    blob.upload_from_file(file_stream, content_type='application/octet-stream')

    print('File transfer completed for file ID:', id)

print('All files transferred.')

Name: 11.json
ID: 1ZQxtdJSx9Pgu0XM2Zhn8Jts4Mw6J1x9i
Last Modified Date: 2023-05-29T20:53:25.000Z

Name: 10.json
ID: 1UjTwRPYVlx_tqFOPCcRr9WSmNNtSWgnb
Last Modified Date: 2023-05-29T20:37:08.000Z

Name: 7.json
ID: 1RRbDaqhj7mUE63FeBNIZnMr2I6tsdpDl
Last Modified Date: 2023-05-29T20:29:40.000Z

Name: 9.json
ID: 1QjRIUj3UxPAC0gLXmEo21gu_OebwiqBX
Last Modified Date: 2023-02-03T02:42:56.000Z

Name: 8.json
ID: 1dYAvdjpLV9GZFejB2hbsIsNjr0-LKI2b
Last Modified Date: 2023-02-03T02:42:42.000Z

Name: 6.json
ID: 1-FRQ8PgfXal_CgEUDlFNpi4s1X2Ylykt
Last Modified Date: 2023-02-03T02:42:18.000Z

Name: 5.json
ID: 1CgzDqnbNuPhTNQT6ORLzYTCxlVKiwIVF
Last Modified Date: 2023-02-03T02:42:08.000Z

Name: 4.json
ID: 1JOowm6HMeZT4PoHBu2xiSfHfDoWnG1z7
Last Modified Date: 2023-02-03T02:41:56.000Z

Name: 3.json
ID: 1N89VDxNI5HkoR-001wIx1IeoWJ_wHpgu
Last Modified Date: 2023-02-03T02:41:42.000Z

Name: 2.json
ID: 1lGfnJOkwFmF31f1p9pNWRToyr4owOOwe
Last Modified Date: 2023-02-03T02:41:32.000Z

Name: 1.json
ID: 1EN0p4oUggO

## Google Maps Reviews

In [5]:
def get_directory_and_file_names_and_ids(service, directory_id, parent_directory_name=""):
    file_names = []
    file_ids = []

    # Query for files in the current directory
    query = f"'{directory_id}' in parents"
    response = service.files().list(q=query).execute()

    # Iterate over the files in the current directory
    for file in response.get('files', []):
        if file['mimeType'] == 'application/vnd.google-apps.folder':
            # If the file is a subdirectory, recursively call the function
            directory_name = parent_directory_name + file['name'] + "-"
            names, ids = get_directory_and_file_names_and_ids(service, file['id'], directory_name)
            file_names.extend(names)
            file_ids.extend(ids)
        else:
            # If the file is not a subdirectory, add the combined name and ID to the lists
            combined_name = parent_directory_name + file['name']
            file_names.append(combined_name)
            file_ids.append(file['id'])

    return file_names, file_ids

# ID of the top-level directory
top_directory_id = '1DiwjU9h_L2tW-FpLj5N4wbnPfmGlfUgc'

# Call the function to retrieve directory and file names and IDs recursively
names, ids = get_directory_and_file_names_and_ids(drive_service, top_directory_id)

# Print the directory and file names
# for name, file_id in zip(names, ids):
#     print("Name:", name)
#     print("ID:", file_id)
#     print()

# Alternatively, you can access the directory and file names and IDs separately as lists
print("Directory and File Names:", names)
print("File IDs:", ids)

# Create a client for Google Cloud Storage directly using the credentials
client = storage.Client(credentials=credentials, project=project_id)

# Define the target directory path within the bucket
target_directory = "gmap_reviews/"

# Iterate over the file URLs and names simultaneously
for id, file_name in zip(ids, names):
    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=id)
    file_stream = io.BytesIO()
    downloader = MediaIoBaseDownload(file_stream, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()

    # Upload the file to Google Cloud Storage in the target directory
    target_blob_name = target_directory + file_name
    blob = client.bucket(bucket_name).blob(target_blob_name)
    file_stream.seek(0)  # Reset the stream position to the beginning
    blob.upload_from_file(file_stream, content_type='application/octet-stream')

    print('File transfer completed for file ID:', id)

print('All files transferred.')

Directory and File Names: ['review-Vermont-2.json', 'review-Vermont-3.json', 'review-Vermont-1.json', 'review-Massachusetts-16.json', 'review-Massachusetts-14.json', 'review-Massachusetts-13.json', 'review-Massachusetts-15.json', 'review-Massachusetts-12.json', 'review-Massachusetts-11.json', 'review-Massachusetts-10.json', 'review-Massachusetts-9.json', 'review-Massachusetts-8.json', 'review-Massachusetts-7.json', 'review-Massachusetts-6.json', 'review-Massachusetts-5.json', 'review-Massachusetts-3.json', 'review-Massachusetts-4.json', 'review-Massachusetts-2.json', 'review-Massachusetts-1.json', 'review-Michigan-15.json', 'review-Michigan-14.json', 'review-Michigan-13.json', 'review-Michigan-12.json', 'review-Michigan-11.json', 'review-Michigan-10.json', 'review-Michigan-9.json', 'review-Michigan-8.json', 'review-Michigan-7.json', 'review-Michigan-6.json', 'review-Michigan-5.json', 'review-Michigan-4.json', 'review-Michigan-3.json', 'review-Michigan-2.json', 'review-Michigan-1.json',

# business.csv

In [6]:
# Define the path pattern for the Pickle file
yelp_business_file_path = 'gs://bucket123321/business.csv'
yelp_business = pd.read_csv(yelp_business_file_path)

index_to_drop = 14
yelp_business = yelp_business.iloc[:, :index_to_drop]

In [7]:
# Define the output CSV file path
output_csv_path = 'gs://bucket123321/yelp_clean_data/yelp_business.csv'

# Write the DataFrame to a CSV file
yelp_business.to_csv(output_csv_path, index=False)

In [8]:
!bq load --autodetect --source_format=CSV conjunto_testing.yelp_business gs://bucket123321/yelp_clean_data/yelp_business.csv

Waiting on bqjob_r67f115b1c21eba6f_00000188c10c614d_1 ... (6s) Current status: DONE   


## Machine Learning Yelp business

In [9]:
# lista_resultante = ['Acne Treatment', 'Aestheticians', 'Art Restoration', 'Artificial Turf', 'Barbers', 'Beauty & Spas', 'Boudoir Photography', 'Body Contouring', 'Body Shops', 'Chiropractors', 'Clothing Rental', 'Cosmetic Dentists', 'Cosmetic Surgeons', 'Cosmetics & Beauty Supply', 'Cosmetology Schools', 'Costume', 'Country Dance Halls', 'Creperies', 'Cryotherapy', 'Customized Merchandise', 'Dance Studios', 'Dance Wear', 'Day Spas', 'Delicatessen', 'Dermatologists', 'Dietitians', 'Dim Sum', 'Dry Cleaning', 'Dry Cleaning & Laundry', 'Ear Nose & Throat', 'Editorial Services', 'Egyptian', 'Electrolysis Hair Removal', 'Embroidery & Crochet', 'Estheticians', 'Eyebrow Services', 'Eyelash Service', 'Eyewear & Opticians', 'Face Painting', 'Facial Rejuvenation', 'Family Practice', 'Fashion', 'Fitness & Instruction', 'Fitness/Exercise Equipment', 'Florists', 'Flowers & Gifts', 'Formal Wear', 'French', 'General Dentistry', 'Glass & Mirrors', 'Greek', 'Hair Extensions', 'Hair Loss Centers', 'Hair Removal', 'Hair Salons', 'Hair Stylists',  'Hats', 'Henna Artists', 'Holistic Animal Care', 'Beauty Salons', 'Cosmetics & Beauty Supply', 'Day Spas', 'Eyebrow Services', 'Eyelash Service', 'Hair Removal', 'Hair Salons', 'Makeup Artists', 'Massage', 'Medical Spas', 'Nail Salons', 'Perfume', 'Skin Care', 'Tanning Salons', 'Tattoo', 'Waxing', 'Nail Salons', 'Nail Technicians', 'Naturopathic/Holistic', 'Nutritionists', 'Hair Salons', 'Hair Extensions', 'Hair Removal', 'Hair Stylists', 'Hair Loss Centers', 'Makeup Artists', 'Cosmetics & Beauty Supply', 'Skin Care', 'Day Spas', 'Medical Spas', 'Tanning', 'Tattoo', 'Piercing', 'Barbers', 'Massage Therapy', 'Massage', 'Eyebrow Services', 'Eyelash Service', 'Waxing', 'Hair Replacement', 'Wigs', 'Perfume', 'Beauty Supply', 'Acupuncture', 'Chiropractors', 'Reflexology', 'Permanent Makeup', 'Tattoo Removal', 'Weight Loss Centers', 'Pilates', 'Dance Studios', 'Fitness & Instruction', 'Gyms', 'Trainers', 'Saunas', 'Meditation Centers', 'Float Spa', 'Cryotherapy', 'Spray Tanning', 'Piercing', 'Medical Spas', 'Medical Aesthetics', 'Medical Foot Care', 'Medical Supplies', 'Medical Transportation', 'Permanent Makeup', 'Perfume', 'Personal Care Services', 'Photographers', 'Photography Classes', 'Photography Stores & Services', 'Physical Therapy', 'Piercing', 'Pilates', 'Plastic Surgeons', 'Skin Care', 'Spray Tanning', 'Sunglasses', 'Surgeons', 'Swimwear', 'Tanning', 'Tanning Beds', 'Tattoo', 'Tattoo Removal', 'Teeth Whitening', 'Traditional Clothing', 'Traffic Ticketing Law', 'Uniforms', 'Video/Film Production', 'Videographers', 'Vintage & Consignment', 'Vitamins & Supplements', 'Waxing', 'Weight Loss Centers', 'Wigs', "Women's Clothing"]

# yelp_business_ML = yelp_business.dropna(subset=['categories'])
# yelp_business_ML['categories'] = yelp_business_ML['categories'].str.split(', ').str[0]
# yelp_business_ML = yelp_business_ML[yelp_business_ML['categories'].isin(lista_resultante)]

# # Eliminar la columna original 'categories'
# yelp_business_ML.drop(columns=["business_id"], inplace=True)

# # Restablecer el índice del DataFrame
# yelp_business_ML = yelp_business_ML.reset_index(drop=True)

# yelp_business_ML.head(7)

# # Define the output CSV file path
# output_csv_path = 'gs://bucket123321/gmap_clean_data/gonzalo_beauty/yelp_business_ML.csv'

# # Write the DataFrame to a CSV file
# yelp_business_ML.to_csv(output_csv_path, index=False)

# review.json

In [10]:
!bq load --autodetect --source_format=NEWLINE_DELIMITED_JSON conjunto_testing.yelp_review gs://bucket123321/review.json

Waiting on bqjob_r743ca5ef243d9048_00000188c10c8b65_1 ... (22s) Current status: DONE   


# user.parquet

In [11]:
# Define the path pattern for the Parquet file
yelp_user_file_path = 'gs://bucket123321/user.parquet'

# Read the Parquet files into a DataFrame
yelp_user = pd.read_parquet(yelp_user_file_path)

In [12]:
# Drop duplicates from the DataFrame
yelp_user = yelp_user.drop_duplicates()

In [13]:
# Define the output CSV file path
output_csv_path = 'gs://bucket123321/yelp_clean_data/user.csv'

# Write the DataFrame to a CSV file
yelp_user.to_csv(output_csv_path, index=False)

In [14]:
!bq load --autodetect --source_format=CSV conjunto_testing.yelp_user gs://bucket123321/yelp_clean_data/user.csv

Waiting on bqjob_r5aead7aa16dbc915_00000188c10f82b2_1 ... (22s) Current status: DONE   


# gmap_metadata

In [15]:
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Read the JSON files into Spark DataFrames
gmaps_metadata = spark.read.json('gs://bucket123321/gmap_metadata/*.json')

# Drop columns
columns_to_drop = ["hours", "MISC", "state", "price", "description"]
gmaps_metadata = gmaps_metadata.drop(*columns_to_drop)

# Remove duplicates from the DataFrame
gmaps_metadata = gmaps_metadata.dropDuplicates()

gmaps_metadata.show()

[Stage 1:====================================================>    (21 + 2) / 23]

+--------------------+----------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+--------------------+--------------------+
|             address|avg_rating|            category|             gmap_id|          latitude|          longitude|                name|num_of_reviews|    relative_results|                 url|
+--------------------+----------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+--------------------+--------------------+
|Professional Marb...|       5.0|[General contract...|0x87b16a1a73c102d...|         35.996193|         -96.763859|Professional Marb...|             3|[0x87b10c52fa736f...|https://www.googl...|
|Hot Box, 1-99 Kin...|       4.2|        [Restaurant]|0x8890b9241e70466...|30.391411299999998| -87.26722029999999|             Hot Box|             5|                null|https://www.googl...|
|Southern Eyecare,...|       5.0|  

In [16]:
# Define the output JSON file path
output_json_path = 'gs://bucket123321/gmap_clean_data/metadata'

# Repartition the DataFrame to a single partition
gmaps_metadata = gmaps_metadata.repartition(1)

# Write the DataFrame to a JSON file 
gmaps_metadata.write.json(output_json_path)

In [17]:
!bq load --autodetect --source_format=NEWLINE_DELIMITED_JSON conjunto_testing.gmaps_metadata gs://bucket123321/gmap_clean_data/metadata/*.json

Waiting on bqjob_r1e22e2e17c75f414_00000188c1127406_1 ... (22s) Current status: DONE   


## Machine Learning gmaps metadata

In [18]:
# word_filter = ['Shaving Ice', 'Weight Loss Centers', 'Eyelash salon', 'Barbershop', 'Nail salon', 'Tanning', 'Reflexology', "Women's clothing store", 'Perfume store', 'Massage', 'Massage Therapy', 'Spray Tanning',
#                     'Beauty', 'Cosmetology Schools', 'Eyebrow Services', 'Shaved Snow', 'Jewelry engraver', 'Hair Extensions', 'Clothing alteration service', 'Skin care clinic', 'Nail Technicians', 'Beauty school', 'Day spa',
#                     'Cosmetics store', 'Rolfing', 'Musical instrument repair shop', 'Shoe repair shop', 'Hair Stylists', 'Permanent Makeup', 'Tanning salon', 'Threading Services', 'Beauty product supplier', 'Beauty & Spas',
#                     'Medical Spas', 'Beauty salon', 'Makeup artist', 'Photo shop', 'Boot repair shop', 'Blow Dry/Out Services', 'Body Contouring', 'Saunas', 'Hair removal service', 'Hair salon', 'Tattoo and piercing shop',
#                     'Electrolysis hair removal service', 'Massage studio', 'Naturopathic/Holistic', 'Orthodontist', 'Spa', 'Clothing store', 'Makeup Artists', 'Beauty products wholesaler', 'Tattoo', 'Skin Care', 'Halotherapy',
#                     'Facial spa', 'Sewing machine repair service', 'Quilt shop', 'Hair Removal', 'Eyelash Service', 'Hair Salons', 'Permanent make-up clinic', 'Cosmetics & Beauty Supply', 'Eyelash service', 'Piercing',
#                     'Cosmetic dentist', 'Wig shop', 'Nail Salons', 'Camera store', 'Waxing']

# # Create a UDF to extract the first word from each array element
# extract_first_word = udf(lambda array: array[0] if array else None, StringType())

# # Extract the first word from the category array
# gmaps_metadata = gmaps_metadata.withColumn("first_word", extract_first_word(gmaps_metadata.category))

# # Filter the rows based on the intersection of first_word and word_filter
# filtered_metadata = gmaps_metadata.filter(gmaps_metadata.first_word.isin(word_filter))

# # Show the filtered DataFrame
# filtered_metadata.show()

# # Define the output JSON file path
# output_json_path = 'gs://bucket123321/gmap_clean_data/gonzalo_beauty'

# # Repartition the DataFrame to a single partition
# filtered_metadata = filtered_metadata.repartition(1)

# # Write the DataFrame to a JSON file 
# filtered_metadata.write.json(output_json_path)

# gmap_reviews

In [19]:
# Create a SparkSession
spark_reviews = SparkSession.builder.getOrCreate()

# Read the JSON files into Spark DataFrames
gmaps_reviews = spark_reviews.read.json('gs://bucket123321/gmap_reviews/*.json')

# Drop the "pics" column
gmaps_reviews = gmaps_reviews.drop("pics")

# Drop the "resp" column
gmaps_reviews = gmaps_reviews.drop("resp")

# Remove duplicates from the DataFrame
gmaps_reviews = gmaps_reviews.dropDuplicates()

# Apply the formatting to the 'time' column
gmaps_reviews = gmaps_reviews.withColumn("formatted_time", from_unixtime(gmaps_reviews["time"]/1000, "yyyy-MM-dd HH:mm:ss"))

# Drop the "time" column
gmaps_reviews = gmaps_reviews.drop("time")

# Show the result
gmaps_reviews.show()

[Stage 10:>                                                         (0 + 1) / 1]

+--------------------+----------------+------+--------------------+--------------------+-------------------+
|             gmap_id|            name|rating|                text|             user_id|     formatted_time|
+--------------------+----------------+------+--------------------+--------------------+-------------------+
|0x8753066fe7a9849...|   0 estrellitas|     5|This place is ama...|11663602073164912...|2018-05-30 22:43:40|
|0x87456251970f27d...|              24|     5|                null|11310259359526237...|2019-06-08 17:20:02|
|0x8752896b1045dd4...|     805inthe801|     5|Great barbershop!...|11390101367485736...|2020-10-20 21:59:58|
|0x876c7d7893d2327...|          A Ames|     5|We had 3-4 contra...|11678322735417736...|2019-06-23 23:33:22|
|0x8713a3c9544d69f...|   A Google User|     5|                null|11159948634753443...|2018-11-01 20:37:32|
|0x876c79f5f9a7206...| A. Covia (Anna)|     5|                null|11505359494553293...|2016-08-27 19:16:51|
|0x875280e84a14a00.

In [20]:
# Define the output JSON file path
output_json_path = 'gs://bucket123321/gmap_clean_data/reviews'

# Repartition the DataFrame to a single partition
gmaps_reviews = gmaps_reviews.repartition(1)

# Write the DataFrame to a JSON file
gmaps_reviews.write.json(output_json_path)

In [21]:
!bq load --autodetect --source_format=NEWLINE_DELIMITED_JSON conjunto_testing.gmaps_reviews gs://bucket123321/gmap_clean_data/reviews/*.json

Waiting on bqjob_r4d94b483be5709bb_00000188c131c57b_1 ... (33s) Current status: DONE   
